## BLS Usual Weekly Earnings Data

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Series stored as a dictionary
series = {'LEU0252911200': 'p10uwe',
          'LEU0252911300': 'p25uwe',
          'LEU0252881500': 'p50uwe',
          'LEU0252911400': 'p75uwe',
          'LEU0252911500': 'p90uwe',
          'LEU0254466800': 'nuwe'}

# Start year and end year
dates = (1988, 2022)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'uwe_main.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### UWE and Growth

In [2]:
columns = ['MONTH', 'YEAR', 'AGE', 'PWORWGT', 'WKEARN', 'HRSUSL1', 'WORKFT']
dfc = (pd.concat([pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
                    .query('WKEARN > 0 and WORKFT == 1')
                  for year in range(1989, 2023)]))
res = dfc.groupby(['YEAR', 'MONTH']).apply(binned_wage).rename('All')
res.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in res.index]
df = res.rolling(3).mean()
df.to_csv(data_dir / 'uwe_bd.csv', index_label='date')
dfgr = (df.pct_change(12).dropna() * 100)
dfgr.to_csv(data_dir / 'uwe_bd_gr.csv', index_label='date')
df2 = (pd.read_csv(data_dir / 'uwe_main.csv', parse_dates=['date'])
         .set_index('date')).p10uwe.dropna()

df2.loc['1989':].to_csv(data_dir / 'uwe_bls.csv', index_label='date')
df2gr = (df2.pct_change(4).dropna() * 100)
df2gr.to_csv(data_dir / 'uwe_bls_gr.csv', index_label='date')

In [3]:
ldate1 = dtxt(df.index[-1])['mon1']
lval1 = df.iloc[-1]
lgrval1 = dfgr.iloc[-1]
prdate1 = dtxt(df.index[-2])['mon1']
prval1 = df.iloc[-2]
prgrval1 = dfgr.iloc[-2]
pr2date1 = dtxt(df.index[-3])['mon1']
pr2val1 = df.iloc[-3]
pr2grval1 = dfgr.iloc[-3]
pr3date1 = dtxt(df.index[-13])['mon1']
pr3val1 = df.iloc[-13]

ldate2 = dtxt(df2.index[-1])['qtr1']
lval2 = df2.iloc[-1]
prdate2 = dtxt(df2.index[-5])['qtr1']
pr2date2 = dtxt(df2.index[-2])['qtr1']
prval2 = df2.iloc[-5]
lgrval2 = df2gr.iloc[-1]
lgr2val2 = df2gr.iloc[-2]
url = 'https://www.bls.gov/webapps/legacy/cpswktab5.htm'
text = (f'BLS \href{{{url}}}{{calculations}} {c_line("blue!65!black")} '+
        f'for {ldate2} show nominal first decile usual weekly earnings '+
        f'of \${lval2:.0f}, compared to \${prval2:.0f} in {prdate2}, '+
        f'resulting in one-year growth of {lgrval2:.1f} percent. In the '+
        f'previous quarter, {pr2date2}, first decile usual weekly '+
        f'earnings grew by {lgr2val2:.1f} percent over the year. '+
        f"Author's calculations from the CPS {c_line('lime!65!green!90!black')} "+
        'show three-month moving average first decile usual weekly '+
        f'earnings of \${lval1:.0f} in {ldate1}, \${prval1:.0f} in '+
        f'{prdate1}, and \${pr3val1:.0f} in {pr3date1}. One-year growth '+
        f'was {lgrval1:.1f} percent for the three months ending {ldate1}, '+
        f'{prgrval1:.1f} percent for the three months ending {prdate1}, '+
        f'and {pr2grval1:.1f} percent for the three months ending '
        f'{pr2date1}.')
write_txt(text_dir / 'uwe_basic.txt', text)
print(text)

BLS \href{https://www.bls.gov/webapps/legacy/cpswktab5.htm}{calculations} (see {\color{blue!65!black}\textbf{---}}) for 2022 Q3 show nominal first decile usual weekly earnings of \$560, compared to \$510 in 2021 Q3, resulting in one-year growth of 9.8 percent. In the previous quarter, 2022 Q2, first decile usual weekly earnings grew by 9.0 percent over the year. Author's calculations from the CPS (see {\color{lime!65!green!90!black}\textbf{---}}) show three-month moving average first decile usual weekly earnings of \$568 in December 2022, \$570 in November 2022, and \$517 in December 2021. One-year growth was 9.9 percent for the three months ending December 2022, 10.9 percent for the three months ending November 2022, and 11.2 percent for the three months ending October 2022.


In [4]:
srs = {'First decile': 'p10uwe', 'First quartile': 'p25uwe', 
       'Median': 'p50uwe', 'Third quartile': 'p75uwe', 
       'Ninth decile': 'p90uwe'}
df3 = (pd.read_csv(data_dir / 'uwe_main.csv', parse_dates=['date'])
        .set_index('date')).loc['2000':, srs.values()].dropna()

final = pd.DataFrame()
for i in [-1, -2, -3, -4, -5, -9, -13, -17, -21]:
    final[dtxt(df3.index[i])['qtr1']] = df3.pct_change(4).iloc[i] * 100

final.index = srs.keys()
final.round(1).to_csv(data_dir / 'wage_dist_bls.tex', sep='&', 
                      lineterminator='\\\ ', quotechar=' ')

final = pd.DataFrame()
for i in [-1, -2, -3, -4, -5, -9, -13, -17, -21]:
    final[dtxt(df3.index[i])['qtr1']] = df3.iloc[i]

final.index = srs.keys()
final.round(0).astype('int').to_csv(data_dir / 'wage_dist_bls2.tex', sep='&', 
                                    lineterminator='\\\ ', quotechar=' ')